In [3]:
import pandas as pd
import numpy as np
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import KFold

ModuleNotFoundError: No module named 'numpy'

In [ ]:
df = pd.read_csv('test.csv')

In [ ]:
X = df.drop('completed', axis=1)
y = df['completed']

In [ ]:
def calculate_fp_rates(X, y, col_ind, col_val):
    model = LogisticRegression()

    kfold = KFold(n_splits=10, shuffle=True, random_state=42)

    aucs = []
    fprs = []

    y = np.array(y)
    X = np.array(X)
    for train_index, test_index in kfold.split(X):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]

        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        auc = roc_auc_score(y_test, y_pred)
        aucs.append(auc)
        
        # Train and evaluate your model on the current fold
        # You can replace this with your model training and evaluation code
        conf_matrix = confusion_matrix(y_test, y_pred)

        is_col_not_0 = X_test[:, col_ind] == col_val
        TP = np.sum((y_test[is_col_not_0] == 1) & (y_pred[is_col_not_0] == 1))
        FN = np.sum((y_test[is_col_not_0] == 1) & (y_pred[is_col_not_0] == 0))
        TN = np.sum((y_test[is_col_not_0] == 0) & (y_pred[is_col_not_0] == 0))
        FP = np.sum((y_test[is_col_not_0] == 0) & (y_pred[is_col_not_0] == 1))      

        # Extract FP and TN from the confusion matrix
        # FP = conf_matrix[0, 1]  # False Positives
        # TN = conf_matrix[1, 1]  # True Negatives

        # Calculate FPR
        FPR = FP / (FP + TN)
        fprs.append(FPR)
        # print("False Positive Rate:", FPR)

    auc_avg = np.average(aucs)
    auc_std = np.std(aucs)
    fpr_avg = np.average(fprs)
    fpr_std = np.std(fprs)
    # print("AUC:", auc_avg, auc_std)
    print("FPR:", fpr_avg, fpr_std)

In [ ]:
calculate_fp_rates(X, y, 11, 1)